In [1]:
# Imports
import gurobipy as gp
from gurobipy import GRB
import ast
import os
import itertools as it
import time 

from railway import *

### Models

In [2]:
# Model 0o
def optimization0o(
    filename,
    # T, 
    # N,
    # N_coordinates,
    # A,
    # w_exp,
    # w_inc,
    # OD,
    # phi,
    # R,
    # W_exp,
    # J,
    # J_a,
    # p,
    # E,
    # C,
    # t_int,
    runtime,
):

    # Gurobi model
    model = gp.Model()

    # Load and set parameters
    open_parameters = []
    with open(filename, "r") as f:
        lines = f.readlines()
        counter = 0
        for line in lines:
            if counter > 0:
                open_parameters.append(ast.literal_eval(line))
            counter += 1
    T,N, N_coordinates, A, w_exp, w_inc, OD, phi,R, W_exp,J,J_a,p, E, C, t_int  = open_parameters

    lambda_e = 0
    k = 3
    M = 100_000
    
    # Search for arcs with no jobs
    arc_no_job = []
    for a in A:
        if len(J_a[a]) == 0:        
            arc_no_job.append(a)
    
    # Search for never used paths
    h_irrel = []
    for (o,d) in OD:
        max_len = {}
        min_len = {}
        for i in range(1,k+1):
            #compute max and min traveltime
            max_len[i] = sum(w_exp[a] if a in arc_no_job else w_inc[a] for a in R[o,d,i])
            min_len[i] = sum(w_exp[a] for a in R[o,d,i])        
        
        for comb in it.combinations(range(1,k+1),2):
            if max_len[comb[0]] < min_len[comb[1]]:
                h_irrel.append((o,d,comb[1]))
            elif max_len[comb[1]] < min_len[comb[0]]:
                h_irrel.append((o,d,comb[0]))
            
    #threshold checking:
    #simple variant without the conversion from track segments to arcs:
    arc_available = []
    for t in range(1,T+1):
        for a in E[t]: #for each track segment in event request area at time t
            if len(J_a[a]) > 0: #if a job could be scheduled
                min_phi = 0
                #check if there is an od pair that certainly uses that arc
                for (o,d) in OD: 
                    if a in R[o,d,1] and a in R[o,d,2] and a in R[o,d,3]:
                        min_phi += phi[o,d,t] #multiply with factor
                        
                if min_phi >= lambda_e:
                    arc_available.append((a,t))
                        

    # Setup
    model_variables = True
    model_constraints = True
    model_objective = True
    model_run = True
    starttime = time.time()
    

    # Variables
    if model_variables:

        y = model.addVars(J,range(1,T+1),vtype=gp.GRB.BINARY, name='y')
        
        x = model.addVars(A,range(1,T+1),vtype=gp.GRB.BINARY, name='x')
        
        h = model.addVars(OD,range(1,T+1),range(1,k+1),vtype=gp.GRB.BINARY, name='h')

        w = model.addVars(A,range(1,T+1),vtype=gp.GRB.CONTINUOUS, name='w')
        
        v = model.addVars(OD,range(1,T+1),vtype=gp.GRB.CONTINUOUS, name='v')
        
    
    # Constraints
    if model_constraints:

        # (2)
        model.addConstrs(
            sum(y[j,t] for t in range(1, T - p[j] + 1)) 
            == 1
            for j in J
        )

        # (3)
        model.addConstrs(
            x[s_1,t_1,t] 
            + sum(
                y[j,t_2] 
                for t_2 in range(max(1,t-p[j]+1),min(t,T)+1)
            ) 
            <= 1  
            for (s_1,t_1) in A 
            for j in J_a[s_1,t_1]  
            for t in range(1,T+1)
        )
    
        # (5)
        model.addConstrs(
            sum(x[s_1,t_1,t] for t in range(1, T+1)) 
            == T -  sum(p[j] for j in J_a[s_1,t_1])
            for (s_1,t_1) in A
        ) 
        
        # (9)
        model.addConstrs(
            sum(h[o,d,t,i] for i in range(1,k+1)) == 1
            for (o,d) in OD
            for t in range(1,T+1)
        )

        # (15)
        model.addConstrs(h[o,d,t,i] == 0 for (o,d,i) in h_irrel for t in range(1,T+1) )
        
        # (10)
        model.addConstrs(
            v[o,d,t] 
            >= sum(w[s_1,t_1,t] for (s_1,t_1) in R[o,d,i]) 
            - M*(1-h[o,d,t,i]) 
            for i in range(1,k+1) 
            for (o,d) in OD 
            for i in range(1,k+1) 
            for t in range(1,T+1)
        )
        
        # (11)
        model.addConstrs(
            v[o,d,t] 
            <= sum(w[s_1,t_1,t] for (s_1,t_1) in R[o,d,i]) 
            for i in range(1,k+1)
            for (o,d) in OD
            for i in range(1,k+1)
            for t in range(1,T+1)
        )
    
        # (4)
        model.addConstrs(
            w[s_1,t_1,t] 
            == w_exp[s_1,t_1]*x[s_1,t_1,t] 
            + w_inc[s_1,t_1]*(1-x[s_1,t_1,t]) 
            for (s_1,t_1) in A 
            for t in range(1,T+1)
        )
        
        # (13)
        model.addConstrs(
            w[s_1,t_1,t] == w_exp[s_1,t_1] 
            for (s_1,t_1) in arc_no_job 
            for t in range(1,T+1)
        )
    
        # (8)
        model.addConstrs(
            sum(
                sum(
                    h[o,d,t,i]*phi[o,d,t] 
                    for i in range(1,k+1) 
                    if (s_1,t_1) in R[o,d,i]
                ) 
                for (o,d) in OD
            )
            <= lambda_e + x[s_1,t_1,t]*M  
            for t in range(1,T+1) 
            for (s_1,t_1) in E[t]
        ) 

        # (12) + (14) bcause arc_available is a bigger set of arc_no_job
        model.addConstrs(
            x[arc[0],arc[1],t] == 1  
            for (arc,t) in arc_available
        )

        # (6)
        # model.addConstrs(
        #     sum((1 - x[s_1,t_1,t]) for (s_1,t_1) in c) <= 1 
        #     for c in C 
        #     for t in range(1,T+1)
        # )
        
        # (7)
        model.addConstrs(
            sum(
                sum(
                    y[j,t_1] 
                    for t_1 in range(max(1,t - p[j] - t_int +1),t+1)
                )
                for j in J_a[a]
            )
            <= 1
            for t in range(1,T+1) 
            for a in A
        )
        
    
    # Objective function
    if model_objective:
        obj = sum(sum((phi[o,d,t]*(v[o,d,t] - W_exp[o,d])) for t in range(1,T+1)) for (o,d) in OD) 
        model.setObjective(obj,GRB.MINIMIZE)


    # Run & Optimize
    if model_run:
        model.setParam('OutputFlag', 0) # verbose
        model.setParam('TimeLimit',runtime)
        model.setParam('LPWarmStart',0)
        model.setParam('PoolSolutions', 1)
        model.params.presolve = 0
        model.params.cuts = 0
        model.params.cutpasses = 0
        model.params.threads = 1
        model.params.heuristics = 0
        model.params.symmetry = 0
        
        model.optimize()
        endtime = time.time()
        # model.printQuality()

    return (
        model.Status,
        model.Runtime,
        0.0,
        endtime - starttime,
        model.MIPgap,
        model.ObjVal,
        model.NodeCount,
        model.IterCount
    )

In [3]:
# Model 0m
def optimization0m(
    T, 
    N,
    N_coordinates,
    A,
    w_exp,
    w_inc,
    OD,
    phi,
    R,
    W_exp,
    J,
    J_a,
    p,
    E,
    C,
    t_int,
    runtime,
):

    # Gurobi model
    model = gp.Model()

    lambda_e = 0
    k = 3
    M = 100_000
    
    # Search for arcs with no jobs
    arc_no_job = []
    for a in A:
        if len(J_a[a]) == 0:        
            arc_no_job.append(a)
    
    # Search for never used paths
    h_irrel = []
    for (o,d) in OD:
        max_len = {}
        min_len = {}
        for i in range(1,k+1):
            #compute max and min traveltime
            max_len[i] = sum(w_exp[a] if a in arc_no_job else w_inc[a] for a in R[o,d,i])
            min_len[i] = sum(w_exp[a] for a in R[o,d,i])        
        
        for comb in it.combinations(range(1,k+1),2):
            if max_len[comb[0]] < min_len[comb[1]]:
                h_irrel.append((o,d,comb[1]))
            elif max_len[comb[1]] < min_len[comb[0]]:
                h_irrel.append((o,d,comb[0]))

    # print('-'*20)
    # print('h_irrel:', len(h_irrel))
    # print(h_irrel)
    # print('-'*20)
            
    #threshold checking:
    #simple variant without the conversion from track segments to arcs:
    arc_available = []
    for t in range(1,T+1):
        for a in E[t]: #for each track segment in event request area at time t
            if len(J_a[a]) > 0: #if a job could be scheduled
                min_phi = 0
                #check if there is an od pair that certainly uses that arc
                for (o,d) in OD: 
                    if a in R[o,d,1] and a in R[o,d,2] and a in R[o,d,3]:
                        min_phi += phi[o,d,t] #multiply with factor
                        
                if min_phi >= lambda_e:
                    arc_available.append((a,t))

    # print('-'*20)
    # print('arc_available:', arc_available)
    # print('-'*20)
                        

    # Setup
    model_variables = True
    model_constraints = True
    model_objective = True
    model_run = True
    starttime = time.time()
    

    # Variables
    if model_variables:

        y = model.addVars(J,range(1,T+1),vtype=gp.GRB.BINARY, name='y')
        
        x = model.addVars(A,range(1,T+1),vtype=gp.GRB.BINARY, name='x')
        
        h = model.addVars(OD,range(1,T+1),range(1,k+1),vtype=gp.GRB.BINARY, name='h')

        w = model.addVars(A,range(1,T+1),vtype=gp.GRB.CONTINUOUS, name='w')
        
        v = model.addVars(OD,range(1,T+1),vtype=gp.GRB.CONTINUOUS, name='v')
        
    
    # Constraints
    if model_constraints:

        # (2)
        c2 = model.addConstrs(
            sum(y[j,t] for t in range(1, T - p[j] + 1)) 
            == 1
            for j in J
        )

        # (3)
        c3 = model.addConstrs(
            x[s_1,t_1,t] 
            + sum(
                y[j,t_2] 
                for t_2 in range(max(1,t-p[j]+1),min(t,T)+1)
            ) 
            <= 1  
            for (s_1,t_1) in A 
            for j in J_a[s_1,t_1]  
            for t in range(1,T+1)
        )
    
        # (5)
        c5 = model.addConstrs(
            sum(x[s_1,t_1,t] for t in range(1, T+1)) 
            == T -  sum(p[j] for j in J_a[s_1,t_1])
            for (s_1,t_1) in A
        ) 
        
        # (9)
        c9 = model.addConstrs(
            sum(h[o,d,t,i] for i in range(1,k+1)) == 1
            for (o,d) in OD
            for t in range(1,T+1)
        )

        
        # (10)
        c10 = model.addConstrs(
            v[o,d,t] 
            >= sum(w[s_1,t_1,t] for (s_1,t_1) in R[o,d,i]) 
            - M*(1-h[o,d,t,i]) 
            for i in range(1,k+1) 
            for (o,d) in OD 
            for i in range(1,k+1) 
            for t in range(1,T+1)
        )
        
        # (11)
        c11 = model.addConstrs(
            v[o,d,t] 
            <= sum(w[s_1,t_1,t] for (s_1,t_1) in R[o,d,i]) 
            for i in range(1,k+1)
            for (o,d) in OD
            for i in range(1,k+1)
            for t in range(1,T+1)
        )
    
        # (4)
        c4 = model.addConstrs(
            w[s_1,t_1,t] 
            == w_exp[s_1,t_1]*x[s_1,t_1,t] 
            + w_inc[s_1,t_1]*(1-x[s_1,t_1,t]) 
            for (s_1,t_1) in A 
            for t in range(1,T+1)
        )
        
        # (13)
        c13 = model.addConstrs(
            w[s_1,t_1,t] == w_exp[s_1,t_1] 
            for (s_1,t_1) in arc_no_job 
            for t in range(1,T+1)
        )
    
        # (8)
        c8 = model.addConstrs(
            sum(
                sum(
                    h[o,d,t,i]*phi[o,d,t] 
                    for i in range(1,k+1) 
                    if (s_1,t_1) in R[o,d,i]
                ) 
                for (o,d) in OD
            )
            <= lambda_e + x[s_1,t_1,t]*M  
            for t in range(1,T+1) 
            for (s_1,t_1) in E[t]
        ) 

        # (12) + (14) bcause arc_available is a bigger set of arc_no_job
        c1214 = model.addConstrs(
            x[arc[0],arc[1],t] == 1  
            for (arc,t) in arc_available
        )

        # (6)
        c6 = model.addConstrs(
            sum((1 - x[s_1,t_1,t]) for (s_1,t_1) in c) <= 1 
            for c in C 
            for t in range(1,T+1)
        )
        
        # (7)
        c7 = model.addConstrs(
            sum(
                sum(
                    y[j,t_1] 
                    for t_1 in range(max(1,t - p[j] - t_int +1),t+1)
                )
                for j in J_a[a]
            )
            <= 1
            for t in range(1,T+1) 
            for a in A
        )

        # (15)
        c15 = model.addConstrs(
            h[o,d,t,i] == 0 
            for (o,d,i) in h_irrel 
            for t in range(1,T+1)
        )

        print('-'*20)
        somma_test = sum(
            1
            for (o,d,i) in h_irrel 
            for t in range(1,T+1)
        )
        print('h_irrel:', len(h_irrel))
        print('sum:', somma_test)
        print('length c15:', len(c15))
        print('-'*20)
        
        constraints_obj = [c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c1214, c13, c1214, c15]
    
    # Objective function
    if model_objective:
        obj = sum(sum((phi[o,d,t]*(v[o,d,t] - W_exp[o,d])) for t in range(1,T+1)) for (o,d) in OD) 
        c1 = model.setObjective(obj,GRB.MINIMIZE)


    # Run & Optimize
    if model_run:
        # model.setParam('OutputFlag', 0) # verbose
        model.setParam('TimeLimit',runtime)
        model.setParam('LPWarmStart',0)
        model.setParam('PoolSolutions', 1)
        model.params.presolve = 0
        model.params.cuts = 0
        model.params.cutpasses = 0
        model.params.threads = 1
        model.params.heuristics = 0
        model.params.symmetry = 0
        
        model.optimize()
        endtime = time.time()
        # model.printQuality()

    return (
        model.Status,
        model.Runtime,
        0.0,
        endtime - starttime,
        model.MIPgap,
        model.ObjVal,
        model.NodeCount,
        model.IterCount,
        constraints_obj,
    )

### Comparison

In [4]:
# Load problem parameters and files

# Define problem parameters
P = 2000
K = 3
timelimit = 60
heuristics_timelimit = 60

# N = 10; J = 10; T = 10 ; ID =  1
# N = 10; J = 10; T = 50 ; ID =  2
# N = 10; J = 10; T = 100; ID =  3
N = 10; J = 40; T = 10 ; ID =  4
# N = 10; J = 40; T = 50 ; ID =  5
# N = 10; J = 40; T = 100; ID =  6
# N = 10; J = 80; T = 10 ; ID =  7
# N = 10; J = 80; T = 50 ; ID =  8
# N = 10; J = 80; T = 100; ID =  9
# N = 20; J = 10; T = 10 ; ID = 10
# N = 20; J = 10; T = 50 ; ID = 11
# N = 20; J = 10; T = 100; ID = 12
# N = 20; J = 40; T = 10 ; ID = 13
# N = 20; J = 40; T = 50 ; ID = 14
# N = 20; J = 40; T = 100; ID = 15
# N = 20; J = 80; T = 10 ; ID = 16
# N = 20; J = 80; T = 50 ; ID = 17
# N = 20; J = 80; T = 100; ID = 18
# N = 40; J = 10; T = 10 ; ID = 19
# N = 40; J = 10; T = 50 ; ID = 20
# N = 40; J = 10; T = 100; ID = 21
# N = 40; J = 40; T = 10 ; ID = 22
# N = 40; J = 40; T = 50 ; ID = 23
# N = 40; J = 40; T = 100; ID = 24
# N = 40; J = 80; T = 10 ; ID = 25
# N = 40; J = 80; T = 50 ; ID = 26
# N = 40; J = 80; T = 100; ID = 27

# Name of the file to load
FILENAME = f"../datasets/railway_N{N}_T{T}_J{J}_P{P}_K{K}.json"
# TESTFILE = f"../datasets/test_N{N}_J{J}_T{T}.txt"

Aname = N*(N-1)//2
TESTFILE = f"../testsets/test_n{N}_A{Aname}_j{J}_T{T}_E0_C0_int2_k3_undirectedarc.txt"

# Display problem parameters
print('Time limit:', timelimit)
print('Heuristics time limit:', heuristics_timelimit)
print(f'ID: {ID}, N: {N}, J: {J}, T: {T}, P: {P}, K: {K}')

Time limit: 60
Heuristics time limit: 60
ID: 4, N: 10, J: 40, T: 10, P: 2000, K: 3


In [5]:
# Model 0: "as-is" Gurobi model with no heuristics or cuts
# print("\nModel 0\n")
model0 = Railway.load(FILENAME)
# model0.model.setParam('OutputFlag', 0) # verbose
model0.model.setParam('TimeLimit', timelimit) # time limit
model0.model.setParam('LPWarmStart', 0)
model0.model.setParam('PoolSolutions', 1)
model0.model.setParam('Cuts', 0)
model0.model.setParam('CutPasses', 0)
model0.model.setParam('Heuristics', 0)
model0.model.setParam('Symmetry', 0)
model0.model.setParam('Threads', 1)
model0.model.setParam('Presolve', 0)

vincoli0 = model0.set_constraints()
model0.set_objective()

results0 = model0.optimize()

# Live results
print()
print(
    f" Model".ljust(6),
    f"| ID".ljust(4),
    f"| status".ljust(13),
    f"| runtime".ljust(10),
    f"| heuristics".ljust(12),
    f"| total".ljust(10),
    f"| gap".ljust(10),
    f"| objective".ljust(12),
    f"| nodes".ljust(10),
)
print("-" * 95)
print(
    f"   {0}".ljust(6),
    f"| {ID}".ljust(4),
    f"| {model0.get_status()}".ljust(13),
    f"| {results0['runtime']:.2f}".ljust(10),
    f"| {0:.2f}".ljust(12),
    f"| {results0['runtime']:.2f}".ljust(10),
    f"| {100*results0['gap']:.2f}%".ljust(10),
    f"| {results0['obj']:.2e}".ljust(12),
    f"| {int(results0['nodes']):d}".ljust(10),
)

Set parameter Username
Set parameter LicenseID to value 2629256
Academic license - for non-commercial use only - expires 2026-02-27
Set parameter TimeLimit to value 60
Set parameter LPWarmStart to value 0
Set parameter PoolSolutions to value 1
Set parameter Cuts to value 0
Set parameter CutPasses to value 0
Set parameter Heuristics to value 0
Set parameter Symmetry to value 0
Set parameter Threads to value 1
Set parameter Presolve to value 0
--------------------
Never used routes: 162
sum: 1620
--------------------
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Non-default parameters:
TimeLimit  60
LPWarmStart  0
Heuristics  0
Symmetry  0
Cuts  0
CutPasses  0
Presolve  0
Threads  1
PoolSolutions  1

Optimize a model with 9445 rows, 4900 columns and 42741 nonzeros
Model fingerprint: 0x8f9394f7
Variab

In [6]:
def count_constraints_by_name(model, name_prefix):
    count = 0
    for constr in model.getConstrs():
        if constr.ConstrName.startswith(name_prefix):
            count += 1
    return count

# Example usage
Ccount = {}
Crange = range(2, 16)
for i in Crange:
    Ccount[i] = count_constraints_by_name(model0.model, str(i))
    print(f"Number of constraints named '{i}': {Ccount[i]}")

# Test again just to be sure
# num_constraints_15 = count_constraints_by_name(model0.model, "15")
# print(f"Number of constraints named '15': {num_constraints_15}")

Number of constraints named '2': 40
Number of constraints named '3': 400
Number of constraints named '4': 450
Number of constraints named '5': 45
Number of constraints named '6': 0
Number of constraints named '7': 450
Number of constraints named '8': 0
Number of constraints named '9': 900
Number of constraints named '10': 2700
Number of constraints named '11': 2700
Number of constraints named '12': 0
Number of constraints named '13': 140
Number of constraints named '14': 0
Number of constraints named '15': 1610


In [10]:
for i, c in enumerate(vincoli0):
    if c is not None:
        print(f"Number of constraints named '{i+2}': {len(c)}")
    else:
        print(f"Number of constraints named '{i+2}': {0}")


Number of constraints named '2': 40
Number of constraints named '3': 400
Number of constraints named '4': 450
Number of constraints named '5': 45
Number of constraints named '6': 0
Number of constraints named '7': 450
Number of constraints named '8': 0
Number of constraints named '9': 900
Number of constraints named '10': 2700
Number of constraints named '11': 2700
Number of constraints named '12': 0
Number of constraints named '13': 140
Number of constraints named '14': 0
Number of constraints named '15': 1610


In [ ]:
# # Model 0o
# (
#     status0,
#     runtime0,
#     heuristics_time0,
#     total_time0,
#     gap0,
#     objective0,
#     nodes0,
#     iterations0
# ) = optimization0o(TESTFILE, timelimit)

# # Live results
# print()
# print(
#     f" Model".ljust(6),
#     f"| ID".ljust(4),
#     f"| status".ljust(13),
#     f"| runtime".ljust(10),
#     f"| heuristics".ljust(12),
#     f"| total".ljust(10),
#     f"| gap".ljust(10),
#     f"| objective".ljust(12),
#     f"| nodes".ljust(10),
# )
# print("-" * 95)
# print(
#     f"   {0}".ljust(6),
#     f"| {1}o".ljust(4),
#     f"| {status0}".ljust(13),
#     f"| {runtime0:.2f}".ljust(10),
#     f"| {0:.2f}".ljust(12),
#     f"| {total_time0:.2f}".ljust(10),
#     f"| {100*gap0:.2f}%".ljust(10),
#     f"| {objective0:.2e}".ljust(12),
#     f"| {int(nodes0):d}".ljust(10),
# )


 Model | ID | status      | runtime  | heuristics | total    | gap      | objective  | nodes   
-----------------------------------------------------------------------------------------------
   0   | 1o | 2           | 0.53     | 0.00       | 1.75     | 0.00%    | 3.08e+04   | 3       


In [6]:
# Prepare inputs from modified model

# Coords
coords_dict = {}
for i in range(1, len(model0.coords)+1):
    coords_dict[i] = model0.coords[i-1]

# E
E = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: []}
# E_list = {}
# for key, value in model0.E.items():
#     E_list[key] = list(value.values())
# print(E_list)

# Model 0m
(
    status0m,
    runtime0m,
    heuristics_time0m,
    total_time0m,
    gap0m,
    objective0m,
    nodes0m,
    iterations0m,
    constraints_obj0m,
) = optimization0m(
    T = model0.Tend, 
    N = tuple(model0.N),
    N_coordinates = coords_dict,
    A = model0.A,
    w_exp = model0.omega_e,
    w_inc = model0.omega_j,
    OD = model0.OD,
    phi = model0.phi,
    R = model0.R,
    W_exp = model0.Omega,
    J = list(model0.J),
    J_a = model0.Ja,
    p = model0.pi,
    E = E,
    C = model0.C,
    t_int = 2,
    runtime = timelimit,
)

# Live results
print()
print(
    f" Model".ljust(6),
    f"| ID".ljust(4),
    f"| status".ljust(13),
    f"| runtime".ljust(10),
    f"| heuristics".ljust(12),
    f"| total".ljust(10),
    f"| gap".ljust(10),
    f"| objective".ljust(12),
    f"| nodes".ljust(10),
)
print("-" * 95)
print(
    f"   {0}".ljust(6),
    f"| {1}o".ljust(4),
    f"| {status0m}".ljust(13),
    f"| {runtime0m:.2f}".ljust(10),
    f"| {0:.2f}".ljust(12),
    f"| {total_time0m:.2f}".ljust(10),
    f"| {100*gap0m:.2f}%".ljust(10),
    f"| {objective0m:.2e}".ljust(12),
    f"| {int(nodes0m):d}".ljust(10),
)


--------------------
h_irrel: 230
sum: 2300
length c15: 1620
--------------------
Set parameter TimeLimit to value 60
Set parameter LPWarmStart to value 0
Set parameter PoolSolutions to value 1
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Set parameter CutPasses to value 0
Set parameter Threads to value 1
Set parameter Heuristics to value 0
Set parameter Symmetry to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Non-default parameters:
TimeLimit  60
LPWarmStart  0
Heuristics  0
Symmetry  0
Cuts  0
CutPasses  0
Presolve  0
Threads  1
PoolSolutions  1

Optimize a model with 20925 rows, 4900 columns and 111221 nonzeros
Model fingerprint: 0xec138db9
Variable types: 1350 continuous, 3550 integer (3550 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+05]
  Objective

In [7]:
for i, c in enumerate(constraints_obj0m):
    print(f"Number of constraints named '{i+2}': {len(c)}")

Number of constraints named '2': 40
Number of constraints named '3': 400
Number of constraints named '4': 450
Number of constraints named '5': 45
Number of constraints named '6': 0
Number of constraints named '7': 450
Number of constraints named '8': 0
Number of constraints named '9': 900
Number of constraints named '10': 2700
Number of constraints named '11': 2700
Number of constraints named '12': 0
Number of constraints named '13': 140
Number of constraints named '14': 0
Number of constraints named '15': 1620


### Conversion Check

In [8]:
import ast

open_parameters = []
with open(TESTFILE, "r") as f:
    lines = f.readlines()
    counter = 0
    for line in lines:
        if counter > 0:
            open_parameters.append(ast.literal_eval(line))
        counter += 1
T,N, N_coordinates, A, w_exp, w_inc, OD, phi,R, W_exp,J,J_a,p, E, C, t_int  = open_parameters


In [9]:

print(T)
print(model0.Tend)

print(T==model0.Tend)

10
10
True


In [10]:

print(N)
print(tuple(model0.N))

print(N==tuple(model0.N))

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)
True


In [11]:

print(N_coordinates)

coords_dict = {}
for i in range(1, len(model0.coords)+1):
    coords_dict[i] = model0.coords[i-1]
print(coords_dict)

print(N_coordinates==coords_dict)

{1: (1.0, 0.0), 2: (0.8090169943749475, 0.5877852522924731), 3: (0.30901699437494745, 0.9510565162951535), 4: (-0.30901699437494734, 0.9510565162951536), 5: (-0.8090169943749473, 0.5877852522924732), 6: (-1.0, 1.2246467991473532e-16), 7: (-0.8090169943749475, -0.587785252292473), 8: (-0.30901699437494756, -0.9510565162951535), 9: (0.30901699437494723, -0.9510565162951536), 10: (0.8090169943749473, -0.5877852522924734)}
{1: (1.0, 0.0), 2: (0.8090169943749475, 0.5877852522924731), 3: (0.30901699437494745, 0.9510565162951535), 4: (-0.30901699437494734, 0.9510565162951536), 5: (-0.8090169943749473, 0.5877852522924732), 6: (-1.0, 1.2246467991473532e-16), 7: (-0.8090169943749475, -0.587785252292473), 8: (-0.30901699437494756, -0.9510565162951535), 9: (0.30901699437494723, -0.9510565162951536), 10: (0.8090169943749473, -0.5877852522924734)}
True


In [12]:

print(A)
print(model0.A)


[(5, 9), (4, 7), (1, 3), (6, 9), (4, 8), (5, 6), (2, 8), (8, 9), (1, 6), (3, 7), (2, 5), (5, 8), (1, 2), (6, 7), (4, 9), (2, 9), (3, 10), (6, 10), (8, 10), (1, 5), (3, 6), (7, 10), (1, 10), (3, 4), (4, 10), (2, 6), (4, 5), (1, 4), (2, 10), (9, 10), (3, 9), (2, 3), (1, 9), (3, 5), (2, 7), (7, 9), (5, 10), (4, 6), (6, 8), (5, 7), (3, 8), (1, 8), (1, 7), (7, 8), (2, 4)]
[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 7), (6, 8), (6, 9), (6, 10), (7, 8), (7, 9), (7, 10), (8, 9), (8, 10), (9, 10)]


In [14]:

print(w_exp)
print(model0.omega_e)

for arc in model0.omega_e:
    print(f'{arc}'.ljust(10)+f': {w_exp[arc]:.4f} | {model0.omega_e[arc]:.4f} | diff: {w_exp[arc]-model0.omega_e[arc]:.2e}')


{(1, 2): 0.62, (1, 3): 1.18, (1, 4): 1.62, (1, 5): 1.9, (1, 6): 2.0, (1, 7): 1.9, (1, 8): 1.62, (1, 9): 1.18, (1, 10): 0.62, (2, 3): 0.62, (2, 4): 1.18, (2, 5): 1.62, (2, 6): 1.9, (2, 7): 2.0, (2, 8): 1.9, (2, 9): 1.62, (2, 10): 1.18, (3, 4): 0.62, (3, 5): 1.18, (3, 6): 1.62, (3, 7): 1.9, (3, 8): 2.0, (3, 9): 1.9, (3, 10): 1.62, (4, 5): 0.62, (4, 6): 1.18, (4, 7): 1.62, (4, 8): 1.9, (4, 9): 2.0, (4, 10): 1.9, (5, 6): 0.62, (5, 7): 1.18, (5, 8): 1.62, (5, 9): 1.9, (5, 10): 2.0, (6, 7): 0.62, (6, 8): 1.18, (6, 9): 1.62, (6, 10): 1.9, (7, 8): 0.62, (7, 9): 1.18, (7, 10): 1.62, (8, 9): 0.62, (8, 10): 1.18, (9, 10): 0.62}
{(1, 2): 0.6180339887498948, (1, 3): 1.1755705045849463, (1, 4): 1.618033988749895, (1, 5): 1.902113032590307, (1, 6): 2.0, (1, 7): 1.902113032590307, (1, 8): 1.618033988749895, (1, 9): 1.1755705045849465, (1, 10): 0.6180339887498951, (2, 3): 0.6180339887498948, (2, 4): 1.1755705045849465, (2, 5): 1.618033988749895, (2, 6): 1.902113032590307, (2, 7): 2.0, (2, 8): 1.9021130

In [15]:

print(w_inc)
print(model0.omega_j)

for arc in model0.omega_j:
    print(f'{arc}'.ljust(10)+f': {w_inc[arc]:.4f} | {model0.omega_j[arc]:.4f} | diff: {w_inc[arc]-model0.omega_j[arc]:.2e}')


{(1, 2): 0.806, (1, 3): 1.534, (1, 4): 2.1060000000000003, (1, 5): 2.4699999999999998, (1, 6): 2.6, (1, 7): 2.4699999999999998, (1, 8): 2.1060000000000003, (1, 9): 1.534, (1, 10): 0.806, (2, 3): 0.806, (2, 4): 1.534, (2, 5): 2.1060000000000003, (2, 6): 2.4699999999999998, (2, 7): 2.6, (2, 8): 2.4699999999999998, (2, 9): 2.1060000000000003, (2, 10): 1.534, (3, 4): 0.806, (3, 5): 1.534, (3, 6): 2.1060000000000003, (3, 7): 2.4699999999999998, (3, 8): 2.6, (3, 9): 2.4699999999999998, (3, 10): 2.1060000000000003, (4, 5): 0.806, (4, 6): 1.534, (4, 7): 2.1060000000000003, (4, 8): 2.4699999999999998, (4, 9): 2.6, (4, 10): 2.4699999999999998, (5, 6): 0.806, (5, 7): 1.534, (5, 8): 2.1060000000000003, (5, 9): 2.4699999999999998, (5, 10): 2.6, (6, 7): 0.806, (6, 8): 1.534, (6, 9): 2.1060000000000003, (6, 10): 2.4699999999999998, (7, 8): 0.806, (7, 9): 1.534, (7, 10): 2.1060000000000003, (8, 9): 0.806, (8, 10): 1.534, (9, 10): 0.806}
{(1, 2): 0.8034441853748633, (1, 3): 1.5282416559604302, (1, 4): 

In [16]:

print(OD)
print(model0.OD)

print(OD==model0.OD)

[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 7), (6, 8), (6, 9), (6, 10), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 8), (7, 9), (7, 10), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 9), (8, 10), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 10), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9)]
[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (

In [17]:

print(phi)
print(model0.phi)

print(phi==model0.phi)

{(1, 2, 1): 1303, (1, 2, 2): 793, (1, 2, 3): 222, (1, 2, 4): 136, (1, 2, 5): 1078, (1, 2, 6): 1803, (1, 2, 7): 1135, (1, 2, 8): 467, (1, 2, 9): 1173, (1, 2, 10): 769, (1, 3, 1): 12, (1, 3, 2): 857, (1, 3, 3): 224, (1, 3, 4): 767, (1, 3, 5): 1304, (1, 3, 6): 1142, (1, 3, 7): 1990, (1, 3, 8): 929, (1, 3, 9): 1464, (1, 3, 10): 1023, (1, 4, 1): 1573, (1, 4, 2): 1704, (1, 4, 3): 839, (1, 4, 4): 593, (1, 4, 5): 764, (1, 4, 6): 420, (1, 4, 7): 1673, (1, 4, 8): 21, (1, 4, 9): 872, (1, 4, 10): 1914, (1, 5, 1): 936, (1, 5, 2): 359, (1, 5, 3): 133, (1, 5, 4): 1241, (1, 5, 5): 1593, (1, 5, 6): 944, (1, 5, 7): 445, (1, 5, 8): 156, (1, 5, 9): 64, (1, 5, 10): 1734, (1, 6, 1): 830, (1, 6, 2): 893, (1, 6, 3): 241, (1, 6, 4): 1346, (1, 6, 5): 565, (1, 6, 6): 799, (1, 6, 7): 1833, (1, 6, 8): 1145, (1, 6, 9): 630, (1, 6, 10): 1437, (1, 7, 1): 119, (1, 7, 2): 421, (1, 7, 3): 1131, (1, 7, 4): 855, (1, 7, 5): 810, (1, 7, 6): 525, (1, 7, 7): 1438, (1, 7, 8): 1048, (1, 7, 9): 666, (1, 7, 10): 1520, (1, 8, 1): 

In [18]:

print(R)
print(model0.R)

print(R==model0.R)


{(1, 2, 1): [(1, 6), (2, 6)], (1, 2, 2): [(1, 5), (4, 5), (4, 7), (3, 7), (3, 6), (2, 6)], (1, 2, 3): [(1, 9), (8, 9), (7, 8), (4, 7), (4, 6), (5, 6), (5, 10), (3, 10), (2, 3)], (1, 3, 1): [(1, 10), (2, 10), (2, 8), (7, 8), (3, 7)], (1, 3, 2): [(1, 10), (6, 10), (4, 6), (2, 4), (2, 7), (3, 7)], (1, 3, 3): [(1, 7), (7, 10), (8, 10), (4, 8), (4, 5), (5, 6), (2, 6), (2, 3)], (1, 4, 1): [(1, 2), (2, 8), (5, 8), (5, 10), (6, 10), (4, 6)], (1, 4, 2): [(1, 3), (3, 9), (6, 9), (6, 10), (8, 10), (4, 8)], (1, 4, 3): [(1, 3), (3, 10), (7, 10), (6, 7), (5, 6), (5, 8), (8, 9), (4, 9)], (1, 5, 1): [(1, 7), (3, 7), (3, 5)], (1, 5, 2): [(1, 4), (4, 8), (8, 10), (3, 10), (3, 5)], (1, 5, 3): [(1, 7), (7, 8), (2, 8), (2, 4), (4, 9), (6, 9), (3, 6), (3, 5)], (1, 6, 1): [(1, 6)], (1, 6, 2): [(1, 3), (3, 5), (5, 7), (7, 8), (2, 8), (2, 6)], (1, 6, 3): [(1, 9), (8, 9), (3, 8), (3, 10), (7, 10), (5, 7), (2, 5), (2, 6)], (1, 7, 1): [(1, 7)], (1, 7, 2): [(1, 6), (2, 6), (2, 3), (3, 4), (4, 5), (5, 7)], (1, 7, 3

In [19]:

print(W_exp)
print(model0.Omega)

for arc in model0.Omega:
    print(f'{arc}'.ljust(10)+f': {W_exp[arc]:.4f} | {model0.Omega[arc]:.4f} | diff: {W_exp[arc]-model0.Omega[arc]:.2e}')


{(1, 2): 3.9, (1, 3): 6.219999999999999, (1, 4): 9.22, (1, 5): 4.9799999999999995, (1, 6): 2.0, (1, 7): 1.9, (1, 8): 5.6, (1, 9): 2.24, (1, 10): 3.9, (2, 1): 0.62, (2, 3): 3.9, (2, 4): 8.32, (2, 5): 1.62, (2, 6): 5.32, (2, 7): 2.0, (2, 8): 6.5, (2, 9): 1.62, (2, 10): 3.62, (3, 1): 1.18, (3, 2): 0.62, (3, 4): 0.62, (3, 5): 8.12, (3, 6): 1.62, (3, 7): 1.9, (3, 8): 3.7, (3, 9): 4.9799999999999995, (3, 10): 4.7, (4, 1): 1.62, (4, 2): 6.86, (4, 3): 2.8, (4, 5): 0.62, (4, 6): 3.8600000000000003, (4, 7): 1.7999999999999998, (4, 8): 6.9399999999999995, (4, 9): 2.0, (4, 10): 2.24, (5, 1): 1.9, (5, 2): 1.62, (5, 3): 5.04, (5, 4): 5.760000000000001, (5, 6): 1.7999999999999998, (5, 7): 5.04, (5, 8): 5.94, (5, 9): 1.9, (5, 10): 2.0, (6, 1): 2.52, (6, 2): 2.24, (6, 3): 4.24, (6, 4): 3.62, (6, 5): 0.62, (6, 7): 0.62, (6, 8): 1.24, (6, 9): 6.140000000000001, (6, 10): 1.9, (7, 1): 2.24, (7, 2): 7.66, (7, 3): 3.42, (7, 4): 2.52, (7, 5): 4.26, (7, 6): 5.94, (7, 8): 4.7, (7, 9): 4.32, (7, 10): 3.24, (8, 1

In [20]:

print(J)
print(list(model0.J))

print(J==list(model0.J))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
True


In [21]:

print(J_a)
print(model0.Ja)

print(J_a==model0.Ja)

{(5, 9): [17], (4, 7): [], (1, 3): [], (6, 9): [8, 28], (4, 8): [3, 40], (5, 6): [20], (2, 8): [23], (8, 9): [], (1, 6): [30, 32], (3, 7): [], (2, 5): [], (5, 8): [16, 26], (1, 2): [], (6, 7): [9], (4, 9): [14, 22], (2, 9): [], (3, 10): [15], (6, 10): [], (8, 10): [25], (1, 5): [11], (3, 6): [], (7, 10): [19], (1, 10): [33], (3, 4): [], (4, 10): [6], (2, 6): [21], (4, 5): [7], (1, 4): [24, 39], (2, 10): [], (9, 10): [10, 29], (3, 9): [5], (2, 3): [31], (1, 9): [1, 34], (3, 5): [2], (2, 7): [], (7, 9): [35], (5, 10): [12], (4, 6): [36, 38], (6, 8): [27], (5, 7): [18], (3, 8): [4], (1, 8): [37], (1, 7): [13], (7, 8): [], (2, 4): []}
{(1, 2): [], (1, 3): [], (1, 4): [24, 39], (1, 5): [11], (1, 6): [30, 32], (1, 7): [13], (1, 8): [37], (1, 9): [1, 34], (1, 10): [33], (2, 3): [31], (2, 4): [], (2, 5): [], (2, 6): [21], (2, 7): [], (2, 8): [23], (2, 9): [], (2, 10): [], (3, 4): [], (3, 5): [2], (3, 6): [], (3, 7): [], (3, 8): [4], (3, 9): [5], (3, 10): [15], (4, 5): [7], (4, 6): [36, 38], (4

In [22]:

print(p)
print(model0.pi)

print(p==model0.pi)


{1: 3, 2: 1, 3: 3, 4: 3, 5: 1, 6: 2, 7: 4, 8: 2, 9: 1, 10: 1, 11: 2, 12: 3, 13: 2, 14: 3, 15: 1, 16: 2, 17: 2, 18: 2, 19: 4, 20: 2, 21: 1, 22: 3, 23: 3, 24: 2, 25: 4, 26: 2, 27: 4, 28: 4, 29: 4, 30: 4, 31: 2, 32: 2, 33: 2, 34: 3, 35: 3, 36: 1, 37: 2, 38: 3, 39: 1, 40: 3}
{1: 3, 2: 1, 3: 3, 4: 3, 5: 1, 6: 2, 7: 4, 8: 2, 9: 1, 10: 1, 11: 2, 12: 3, 13: 2, 14: 3, 15: 1, 16: 2, 17: 2, 18: 2, 19: 4, 20: 2, 21: 1, 22: 3, 23: 3, 24: 2, 25: 4, 26: 2, 27: 4, 28: 4, 29: 4, 30: 4, 31: 2, 32: 2, 33: 2, 34: 3, 35: 3, 36: 1, 37: 2, 38: 3, 39: 1, 40: 3}
True


In [23]:

print(E)
print(model0.E)

E_dict = {}
for key, value in E.items():
    E_dict[key] = dict(value)
print(E_dict)

E_list = {}
for key, value in E_dict.items():
    E_list[key] = list(value.values())
print(E_list)

{1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: []}
{}
{1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 10: {}}
{1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: []}


In [24]:

print(C)
print(model0.C)

print(C==model0.C)


[]
[]
True


In [25]:

print(t_int)
print(model0.tau)

2
{(5, 9): 2, (4, 7): 2, (1, 3): 2, (6, 9): 2, (4, 8): 2, (5, 6): 2, (2, 8): 2, (8, 9): 2, (1, 6): 2, (3, 7): 2, (2, 5): 2, (5, 8): 2, (1, 2): 2, (6, 7): 2, (4, 9): 2, (2, 9): 2, (3, 10): 2, (6, 10): 2, (8, 10): 2, (1, 5): 2, (3, 6): 2, (7, 10): 2, (1, 10): 2, (3, 4): 2, (4, 10): 2, (2, 6): 2, (4, 5): 2, (1, 4): 2, (2, 10): 2, (9, 10): 2, (3, 9): 2, (2, 3): 2, (1, 9): 2, (3, 5): 2, (2, 7): 2, (7, 9): 2, (5, 10): 2, (4, 6): 2, (6, 8): 2, (5, 7): 2, (3, 8): 2, (1, 8): 2, (1, 7): 2, (7, 8): 2, (2, 4): 2}
